In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql, magics
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. Yo

In [2]:
import utils

In [3]:
import altair as alt
from dla_utils import _dla_utils

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [4]:
rt_sched = query_sql(
    """
SELECT *
FROM `cal-itp-data-infra-staging.natalie_views.gtfs_rt_vs_sched_routes`
"""
)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [5]:
rt_sched['service_date'] = pd.to_datetime(rt_sched['service_date'])
rt_sched['weekday'] = pd.Series(rt_sched.service_date).dt.day_name()
rt_sched['month'] =  pd.Series(rt_sched.service_date).dt.month_name()

In [6]:
rt_sched.sample(5)

,calitp_itp_id,calitp_url_number,route_id,service_date,num_sched,num_rt,pct_ran,weekday,month
245,300,0,3491,2022-05-27,28,28,1.00,Friday,May
803,300,0,3495,2022-04-19,62,62,1.00,Tuesday,April
349,300,0,3494,2022-06-05,42,36,0.86,Sunday,June
947,300,0,3490,2022-05-02,107,76,0.71,Monday,May
1145,300,0,3479,2022-04-30,131,93,0.71,Saturday,April


In [7]:
(rt_sched
 >>group_by(_.calitp_itp_id,
            _.calitp_url_number,
            _.service_date,
            _.weekday,
           _.month
           )
 >>summarize(total_num_sched = _.num_sched.sum() ,
             total_num_rt = _.num_rt.sum())
 >>mutate(pct_ran = (_.total_num_rt)/(_.total_num_sched))
            )


,calitp_itp_id,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_rt,pct_ran
0,300,0,2022-04-01,Friday,April,1562,1512,0.97
1,300,0,2022-04-02,Saturday,April,871,635,0.73
2,300,0,2022-04-03,Sunday,April,771,556,0.72
3,300,0,2022-04-04,Monday,April,1518,1087,0.72
4,300,0,2022-04-05,Tuesday,April,1616,1482,0.92
...,...,...,...,...,...,...,...,...
86,300,0,2022-06-26,Sunday,June,772,500,0.65
87,300,0,2022-06-27,Monday,June,1465,1017,0.69
88,300,0,2022-06-28,Tuesday,June,1559,1457,0.93
89,300,0,2022-06-29,Wednesday,June,1548,1454,0.94


In [14]:
def by_date(df, sum1_sched, sum2_rt):
    agg_df = (df
     >>group_by(_.calitp_itp_id,
            _.calitp_url_number,
            _.service_date,
            _.weekday,
           _.month)
     >>summarize(total_num_sched = (_[sum1_sched].sum()),
             total_num_rt = (_[sum2_rt].sum()))
     >>mutate(pct_w_rt = (_.total_num_rt)/(_.total_num_sched))
            )
    return agg_df


In [15]:
# cols_to_agg = ['calitp_itp_id', 'calitp_url_number','service_date', 'weekday', 'month']

In [16]:
rt_sched_agg = by_date(rt_sched, 'num_sched', 'num_rt')

In [17]:
rt_sched_agg

,calitp_itp_id,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_rt,pct_w_rt
0,300,0,2022-04-01,Friday,April,1562,1512,0.97
1,300,0,2022-04-02,Saturday,April,871,635,0.73
2,300,0,2022-04-03,Sunday,April,771,556,0.72
3,300,0,2022-04-04,Monday,April,1518,1087,0.72
4,300,0,2022-04-05,Tuesday,April,1616,1482,0.92
...,...,...,...,...,...,...,...,...
86,300,0,2022-06-26,Sunday,June,772,500,0.65
87,300,0,2022-06-27,Monday,June,1465,1017,0.69
88,300,0,2022-06-28,Tuesday,June,1559,1457,0.93
89,300,0,2022-06-29,Wednesday,June,1548,1454,0.94


In [12]:
# checck to see if the function in utils works the same

In [13]:
utils.agg_by_date(rt_sched,'num_sched', 'num_rt')

,calitp_itp_id,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_rt,pct_ran
0,300,0,2022-04-01,Friday,April,1562,1512,0.97
1,300,0,2022-04-02,Saturday,April,871,635,0.73
2,300,0,2022-04-03,Sunday,April,771,556,0.72
3,300,0,2022-04-04,Monday,April,1518,1087,0.72
4,300,0,2022-04-05,Tuesday,April,1616,1482,0.92
...,...,...,...,...,...,...,...,...
86,300,0,2022-06-26,Sunday,June,772,500,0.65
87,300,0,2022-06-27,Monday,June,1465,1017,0.69
88,300,0,2022-06-28,Tuesday,June,1559,1457,0.93
89,300,0,2022-06-29,Wednesday,June,1548,1454,0.94


In [13]:
rt_sched>>count(_.route_id)>>arrange(-_.n)

,route_id,n
0,3479,91
1,3480,91
2,3481,91
4,3485,91
5,3486,91
6,3487,91
8,3489,91
9,3490,91
12,3493,91
13,3494,91


In [14]:
(rt_sched>>filter(_.route_id=='3501')>>arrange(_.service_date))

,calitp_itp_id,calitp_url_number,route_id,service_date,num_sched,num_rt,pct_ran,weekday,month
592,300,0,3501,2022-04-01,54,51,0.94,Friday,April
17,300,0,3501,2022-04-04,104,0,0.00,Monday,April
28,300,0,3501,2022-04-05,104,0,0.00,Tuesday,April
52,300,0,3501,2022-04-06,104,0,0.00,Wednesday,April
43,300,0,3501,2022-04-07,104,0,0.00,Thursday,April
15,300,0,3501,2022-04-08,52,0,0.00,Friday,April
22,300,0,3501,2022-04-18,104,0,0.00,Monday,April
31,300,0,3501,2022-04-19,104,0,0.00,Tuesday,April
25,300,0,3501,2022-04-20,104,0,0.00,Wednesday,April
37,300,0,3501,2022-04-21,104,0,0.00,Thursday,April


In [15]:
(rt_sched>>filter(_.route_id=='3488')
).head()

,calitp_itp_id,calitp_url_number,route_id,service_date,num_sched,num_rt,pct_ran,weekday,month
55,300,0,3488,2022-05-16,6,4,0.67,Monday,May
56,300,0,3488,2022-05-02,6,4,0.67,Monday,May
57,300,0,3488,2022-06-06,6,4,0.67,Monday,June
58,300,0,3488,2022-04-25,6,4,0.67,Monday,April
59,300,0,3488,2022-06-13,6,4,0.67,Monday,June


In [16]:
line = alt.Chart(rt_sched).mark_line().encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_ran', title=_dla_utils.labeling('pct_ran')),
    color='route_id',
    strokeDash='route_id',
)
line_chart = styleguide.preset_chart_config(line)
line_chart = _dla_utils.add_tooltip(line_chart, 'route_id', 'pct_ran')
line_chart.properties(width=600)

alt.Chart(...)

In [17]:
rt_sched>>filter(_.pct_ran<.5)>>count(_.route_id)

,route_id,n
0,3489,1
1,3491,1
2,3501,55


* something funky happening with route 3501. 
* checking trip updates

In [23]:

## seeing what the pct_with_rt would look like without route 3501

In [20]:
no_3501 = utils.agg_by_date((rt_sched>>filter(_.route_id != '3501')),'num_sched', 'num_rt')

In [22]:
no_3501.sample()

,calitp_itp_id,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_rt,pct_ran
31,300,0,2022-05-02,Monday,May,1415,1069,0.76


In [25]:
with_3501 = utils.agg_by_date(rt_sched,'num_sched', 'num_rt')

In [26]:
with_3501.sample()

,calitp_itp_id,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_rt,pct_ran
68,300,0,2022-06-08,Wednesday,June,1610,1416,0.88


In [58]:

line_1 = alt.Chart(with_3501).mark_line().encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_ran', title=_dla_utils.labeling('pct_ran')),
    color=alt.value("#136C97"))
line_2 = alt.Chart(no_3501).mark_line().encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_ran', title=_dla_utils.labeling('pct_ran')),
    color=alt.value("#E16B26"))
chart = line_1+line_2
line_chart = styleguide.preset_chart_config(chart)
line_chart = _dla_utils.add_tooltip(line_chart, 'service_date:O', 'pct_ran:Q')
line_chart.properties(width=600)


alt.LayerChart(...)